In [1]:
from mxnet.gluon import nn
net=nn.Sequential()
with net.name_scope():
    # 第一阶段 
    net.add(nn.Conv2D( channels=224,kernel_size=11,strides=4,activation='relu'))
    net.add(nn.MaxPool2D(pool_size=3,strides=2))
    # 第二阶段 
    net.add(nn.Conv2D(
        channels=256,kernel_size=5,padding=2,activation='relu'))
    net.add(nn.MaxPool2D(pool_size=3,strides=2))
    #第三阶段 
    net.add(nn.Conv2D(
                channels=384,kernel_size=3,padding=1,activation='relu'))
    
    net.add(nn.Conv2D(channels=384,kernel_size=3,padding=1,activation='relu'))
    
    net.add(nn.Conv2D(channels=256,kernel_size=3,padding=1,activation='relu'))
    net.add(nn.MaxPool2D(pool_size=3,strides=2)) 
    # 第四阶段 
    net.add(nn.Flatten())
    net.add(nn.Dense(4096,activation='relu'))
    net.add(nn.Dropout(.5))
    #第五阶段 
    net.add(nn.Dense(4096,activation='relu'))
    net.add(nn.Dropout(.5))
    # 第六阶段 
    net.add(nn.Dense(10))
     
    
    

# 读取数据 
Alexnet 使用的Imagenet数据，其中输入图片大小一般是 224*224。因为imageNet数据训练时间过长，我们用FashionMNIST
来演示。这里我们额外做了一步将数据扩大到224*224 

In [2]:
import sys
sys.path.append('..') 
import utils
from mxnet import image

def transform(data,label):
    data=image.imresize(data,224,224) 
    return utils.transform_mnist(data,label) 
batch_size=64 
train_data,test_data=utils.load_data_fashion_mnist(batch_size) 

# 训练 
这时候我们可以开始训练。相对于前面的LeNet，我们做了如下三个改动：

1. 我们使用Xavier来初始化参数 
2. 使用了更小的学习率
3. 默认只迭代一轮（这样网页编译快点一点 ）

In [3]:
from mxnet import autograd 
from mxnet import gluon 
from mxnet import nd 
from mxnet import init 
ctx=utils.try_gpu()

net.initialize(ctx=ctx,init=init.Xavier())
softmax_cross_entropy=gluon.loss.SoftmaxCrossEntropyLoss()
trainer=gluon.Trainer(
    net.collect_params(),'sgd',{'learning_rate':0.01})
for epoch in range(1):
    train_loss = 0.
    train_acc = 0. 
    for data, label in train_data:
        label=label.as_in_context(ctx)
        with autograd.record():
            output=net(data.as_in_context(ctx ))
            loss=softmax_cross_entropy(output,label)
        loss.backward()
        trainer.step(batch_size) 
        train_loss+=nd.mean(loss).asscalar() 
        train_acc+=utils.accuracy(output,label)
    test_acc=utils.evaluate_accuracy(test_data,net,ctx) 
    print("Epoch %d . loss :%f ,train acc %f,Test acc%f "% 
          (epoch,train_loss/len(train_data),train_acc.asscalar()/len(train_data),test_acc)
          ) 

        

MXNetError: b'[21:53:39] /opt/concourse/worker/volumes/live/e490c994-58a7-4435-729f-6d8cf3f66ffa/volume/libmxnet_1564766672285/work/src/operator/nn/pooling.cc:190: Check failed: param.kernel[0] <= dshape_nchw[2] + 2 * param.pad[0]: kernel size (3) exceeds input (2 padded to 2)\nStack trace:\n  [bt] (0) 1   libmxnet.dylib                      0x000000011186ddd9 dmlc::LogMessageFatal::~LogMessageFatal() + 57\n  [bt] (1) 2   libmxnet.dylib                      0x000000011210f12a mxnet::op::PoolingShape(nnvm::NodeAttrs const&, std::__1::vector<mxnet::TShape, std::__1::allocator<mxnet::TShape> >*, std::__1::vector<mxnet::TShape, std::__1::allocator<mxnet::TShape> >*) + 3978\n  [bt] (2) 3   libmxnet.dylib                      0x00000001119ecc87 mxnet::imperative::SetShapeType(mxnet::Context const&, nnvm::NodeAttrs const&, std::__1::vector<mxnet::NDArray*, std::__1::allocator<mxnet::NDArray*> > const&, std::__1::vector<mxnet::NDArray*, std::__1::allocator<mxnet::NDArray*> > const&, mxnet::Dis